In [1]:
import os
import pathlib
import json
from glob import glob
from PIL import Image

import tensorflow as tf

# Change these paths
VAL_IMG_PATH = os.path.join("/media/david/e021a7c1-e806-43d8-af8c-531486336fba/data/face_detection/WIDER_val/images", '*', '*.jpg')
TRAIN_IMG_PATH = os.path.join("/media/david/e021a7c1-e806-43d8-af8c-531486336fba/data/face_detection/WIDER_train/images", '*', '*.jpg')

def int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def int64_list_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def bytes_list_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def float_list_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

/home/david/.local/share/virtualenvs/tensorflow_models-cADKlZN6/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/david/.local/share/virtualenvs/tensorflow_models-cADKlZN6/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/david/.local/share/virtualenvs/tensorflow_models-cADKlZN6/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint

In [2]:
def get_images(image_path):
    for img_filename in glob(image_path):
        yield img_filename

In [7]:
val_img_files = list(get_images(VAL_IMG_PATH))
train_img_files = list(get_images(TRAIN_IMG_PATH))

In [4]:
def load_image(filename):
    img = Image.open(filename)
    width = img.width
    height = img.height
    with open(filename, 'rb') as f:
        return f.read(), width, height

In [8]:
len(train_img_files)

12880

In [9]:
len(val_img_files)

3226

In [10]:
def load_bbox_data(bbox_def_file):
    bbox_by_image = {}
    with open(bbox_def_file) as f:
        current_filename = None
        bboxes_left_in_run = -1
        current_bboxes = []
        bboxes = []
        for i, line in enumerate(f):
            line = line.strip()
            if line.isnumeric():
                bbox_by_image[current_filename]['n_bboxes'] = int(line)
            elif all(token.isnumeric() for token in line.split()):
                x1, y1, w, h, *rest = line.split()
                bbox = {"x1": int(x1), "y1": int(y1), "x2": int(x1) + int(w), "y2": int(y1) + int(h)}
                bboxes.append(bbox)
            else:
                if current_filename and bboxes:
                    bbox_by_image[current_filename]['bboxes'] = bboxes
                    bboxes = []
                current_filename = line
                bbox_by_image[current_filename] = {}

    return bbox_by_image

In [11]:
# The dataset has detections keyed by the a relative file path, but we have
# absolute paths, so convert those abs paths to the required keys
def get_file_key(file):
    file_name = pathlib.Path(file).name
    parent_dir = pathlib.Path(file).parent.name
    return os.path.join(parent_dir, file_name)

In [13]:
val_bbox = load_bbox_data("/media/david/e021a7c1-e806-43d8-af8c-531486336fba/data/face_detection/WIDER_val/wider_face_val_bbx_gt.txt")

In [14]:
len(list(val_bbox.keys()))

3226

In [16]:
train_bbox = load_bbox_data("/media/david/e021a7c1-e806-43d8-af8c-531486336fba/data/face_detection/WIDER_train/wider_face_train_bbx_gt.txt")

In [17]:
len(list(train_bbox.keys()))

12880

In [18]:
def create_tf_example(image_filename, encoded_img, width, height, bboxes):
  """Creates a tf.Example proto for a given image.

  Args:
    encoded_img: The jpg encoded data of the image.

  Returns:
    example: The created tf.Example.
  """


  image_format = b'jpg'

  xmins = []
  xmaxs = []
  ymins = []
  ymaxs = []
  classes_text = []
  classes = []

  for bbox_data in bboxes:
      # bbox coordinates are normalized
      height = height
      width = width
      box_x1 = bbox_data['x1'] / width
      box_x2 = bbox_data['x2'] / width
      box_y1 = bbox_data['y1'] / height
      box_y2 = bbox_data['y2'] / height
      if box_x1 < 0 or box_x2 > 1.0 or box_y1 < 0 or box_y2 > 1.0:
        continue

      xmins.append(box_x1)
      xmaxs.append(box_x2)
      ymins.append(box_y1)
      ymaxs.append(box_y2)
      classes_text.append(b'face')
      classes.append(1)

  tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': int64_feature(height),
      'image/width': int64_feature(width),
      'image/filename': bytes_feature(bytes(image_filename, encoding='utf-8')),
      'image/source_id': bytes_feature(bytes(image_filename, encoding='utf-8')),
      'image/encoded': bytes_feature(encoded_img),
      'image/format': bytes_feature(image_format),
      'image/object/bbox/xmin': float_list_feature(xmins),
      'image/object/bbox/xmax': float_list_feature(xmaxs),
      'image/object/bbox/ymin': float_list_feature(ymins),
      'image/object/bbox/ymax': float_list_feature(ymaxs),
      'image/object/class/text': bytes_list_feature(classes_text),
      'image/object/class/label': int64_list_feature(classes),
  }))
  return tf_example

In [19]:
def create_dataset(image_filenames, bbox_by_image):
    for image_file in image_filenames:
        encoded_img, width, height = load_image(image_file)
        file_key = get_file_key(image_file)
        try:
            yield create_tf_example(image_file, encoded_img, width, height, bbox_by_image[file_key]['bboxes'])

        except (KeyError, ZeroDivisionError):
            pass

In [20]:
val_dataset = list(create_dataset(val_img_files, val_bbox))

In [21]:
len(val_dataset)

3225

In [22]:
list(val_bbox.values())[0]

{'bboxes': [{'x1': 146, 'x2': 284, 'y1': 134, 'y2': 250},
  {'x1': 398, 'x2': 526, 'y1': 68, 'y2': 224},
  {'x1': 614, 'x2': 744, 'y1': 104, 'y2': 314}],
 'n_bboxes': 3}

In [23]:
def write_dataset(tf_examples, output_path):
  writer = tf.python_io.TFRecordWriter(output_path)

  for tf_example in tf_examples:
    writer.write(tf_example.SerializeToString())

  writer.close()

In [24]:
write_dataset(val_dataset, "/media/david/e021a7c1-e806-43d8-af8c-531486336fba/data/face_detection/WIDER_val/WIDER_val2.tfrecord")

In [28]:
train_dataset = list(create_dataset(train_img_files, train_bbox))

In [30]:
write_dataset(train_dataset, "/media/david/e021a7c1-e806-43d8-af8c-531486336fba/data/face_detection/WIDER_train/WIDER_train2.tfrecord")

In [29]:
len(train_dataset)

12879